In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2

---

# NP probe inserted through V1 and hippo 

In [ ]:
probe = 'Neuropixels'
channel_groups = mz_ena.get_channel_depth(probe)

In [ ]:
root_files = []
matches = [] # list of experiment folders
source_folder = r"G:\Neuropixels\NMDA_V1HPC\SORTED"

for root, dirnames, filenames in os.walk(source_folder):
    for filename in fnmatch.filter(filenames, '*rez.mat'):
        for filename in fnmatch.filter(filenames, '*cluster_group.tsv'):#For newer phy2 GUI, .tsv instead of .csv files
            
            # change this before running otherwise there will be none
            if str('et') in root: 
                if (str('noisy') not in root):
                    matches.append(os.path.join(root, filename))
                    root_files.append(root)
                    print (root)

print('\nIMPORTANT: This has "cluster_group.tsv" already appended to the matches list')
print('How many files?', len(matches))
print('How many root files?', len(root_files))

In [ ]:



all_units_or_good = 1   # if 0--manually sorted good units, if 1--all units from KS




In [ ]:
# run this to check and make sure the splitting in the function below is correct!
f = root_files[0]

f.split('\\')[-1].split('_')

In [ ]:
data_df = []
df_rez = []

for path in root_files:
    cluster_path = os.path.join(path, 'cluster_KSLabel.tsv')    
    #------------------------------------------------------------------------------------
    # These probably change depending on the file naming I used during the recording
    situation = path.split('\\')[-1].split('_')[0]
    group = path.split('\\')[-1].split('_')[1]
    et_num = path.split('\\')[-1].split('_')[2]   # what et
    cc_num = path.split('\\')[-1].split('_')[3]   # what cc
    #------------------------------------------------------------------------------------
    cluster_groups = pd.read_csv(cluster_path, sep = '\t')
    #------------------------------------------------------------------------------------
    if all_units_or_good == 0:
        good = cluster_groups[cluster_groups['group'] == 'good'].cluster_id.values
    elif all_units_or_good == 1:
        good = cluster_groups[cluster_groups['KSLabel'] == 'good'].cluster_id.values
    #------------------------------------------------------------------------------------
    spike_clusters = np.load(os.path.join(path, 'spike_clusters.npy'))
    spike_times = np.load(os.path.join(path, 'spike_times.npy'))
    templates = np.load(os.path.join(path, 'templates.npy'))
    spike_templates = np.load(os.path.join(path, 'spike_templates.npy'))
    #------------------------------------------------------------------------------------
    foo = pd.DataFrame({'situ':situation,
                        'group': group,
                        'et': et_num,
                        'cc':cc_num,
                        'cluster_id':spike_clusters.flatten(),
                        'times':spike_times.flatten()/30000.0, 
                        'templates':spike_templates.flatten(),
                        'path':f})        
    data_df.append(foo)
    #------------------------------------------------------------------------------------
    foo_1 = foo[foo.cluster_id.isin(good)]
    df_rez.append(foo_1)
    #------------------------------------------------------------------------------------
data_df = pd.concat(data_df, axis=0, ignore_index=True)
df_rez = pd.concat(df_rez, axis=0, ignore_index=True)

print('total units df shape:', data_df.shape)
print('"good" units df shape:', df_rez.shape)


In [ ]:
data_df['cuid'] =  data_df.et.astype(str) + str('_') + data_df.cluster_id.astype(str)
df_rez['cuid'] =  df_rez.et.astype(str) + str('_') + df_rez.cluster_id.astype(str)

print("Total units:", data_df['cuid'].nunique())
print("Good units:", df_rez['cuid'].nunique())

df_rez.head()

---

# Keep going from here

In [ ]:
tot_trials = 30

trials_number = tot_trials # ~~~~~~~~~~~~~~~~~~~~~~ IMPORTANT THIS IS CORRECT ~~~~~~~~~~~~~~~~~~~~~~

trial_length = 3.0 #this is the length of the recording in OpenEphys (the yellow highlight)
th_bin = 0.01

ls_rawcount = []
ls_lowspikecount = []
ls_refract_violators = []
ls_lowamp_waveforms = []

### Run these to add a stim column (operant, sf-tuning, ori-tuning)

In [ ]:
# data_df['trial']=(data_df.times//trial_length).astype(int)
# data_df['stim']=data_df.trial.map(dict(zip(np.arange(trials_number),([int(i) for i in overall_order]))))

# df_rez['trial']=(df_rez.times//trial_length).astype(int)
# df_rez['stim']=df_rez.trial.map(dict(zip(np.arange(trials_number),([int(i) for i in overall_order]))))
# df_rez.head()

# For recordings where only 1 stimulus is shown (aka novel recordings, or pre, or post training)
# adding the "stim" column keeps the rest of the code able to run
data_df['trial']=(data_df.times//trial_length).astype(int)
data_df['stim']=0
df_rez['trial']=(df_rez.times//trial_length).astype(int)
df_rez['stim']=0
df_rez.head()

---

# Creating the spikes and tmt dataFrames

In [ ]:
ls_spikes = []
ls_tmt = []

i=0

num_units = df_rez['cuid'].nunique()

for iii, unit in enumerate(df_rez['cuid'].unique()): ##### I changed this from df_rez to data_df to check the units
    cuid = str(unit)
    tmp2 = df_rez[(df_rez.cuid == unit)]             ##### I changed this from df_rez to data_df to check the units
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    cluster_id = tmp2.cluster_id.values[0]
    situ = tmp2.situ.values[0]
    group = tmp2.group.values[0]
    et = tmp2.et.values[0]
    cc = tmp2.cc.values[0]
    path = tmp2.path.values[0]
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    try:
        for stim_id in tmp2.stim.unique():
            tmp3=tmp2[tmp2.stim==stim_id]
            tmt, depth, ch_idx = mz_ena.ksort_get_tmt(tmp3, cluster_id, templates, channel_groups)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            df = mz_ena.getRaster_kilosort(tmp3, unit, trial_length) 
            trials_number_not_empty = len(df.trial.unique())    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            df_spikes_tmp = pd.DataFrame({'cluster_id': cluster_id, 
                                          'spikes': tmp3.times.values,
                                          'trial':df.trial,
                                          'stim_id':stim_id,
                                          'trial_spikes':df.times,
                                          'depth':depth,
                                          'situ':situ,
                                          'group':group,
                                          'et':et,
                                          'cc': cc,
                                          'cuid': cuid,
                                          'path':path})
            ls_spikes.append(df_spikes_tmp)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            df_tmt_tmp = pd.DataFrame({'tmt': tmt,
                                       'stim_id':stim_id,
                                       'depth':depth,
                                       'situ':situ,
                                       'group':group,
                                       'et':et,
                                       'cc': cc,
                                       'cuid': cuid,
                                       'path':path})
            ls_tmt.append(df_tmt_tmp)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    except:
        i+=1
        continue
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    if iii%200 == 0:
        print('done with {0} out of {1}'.format(iii, num_units))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
df_spikes = pd.concat(ls_spikes)
df_tmt = pd.concat(ls_tmt)
print("Total errors: {0} out of {1} units".format(i,num_units))
print("ALL DONE!")

In [ ]:
df_spikes.head(2)

In [ ]:
df_tmt.head(2)

# Function to add metadata

In [ ]:
# def label_group(row):
#     nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
#     if row['et'] in nmda_ls:
#         return "nmda"
#     elif 
#         return "sham"

def label_region(row):
    if row['depth'] <= 1100:
        return 'v1'
    else:
        return 'none'

In [ ]:
# df_spikes['group'] = df_spikes.apply(lambda row: label_group(row), axis=1)
df_spikes['region'] = df_spikes.apply(lambda row: label_region(row), axis=1)

df_spikes.head(2)

In [ ]:
# df_tmt['group'] = df_tmt.apply(lambda row: label_group(row), axis=1)
df_tmt['region'] = df_tmt.apply(lambda row: label_region(row), axis=1)

df_tmt.head(2)

---

# Adding to the waveforms (tmt) dataFrame
Based on Yu's code and outputs the spike width, peak-to-trough amplitude, and the overall rs/fs classification

In [ ]:
import resampy
import scipy
import scipy.signal as ssig

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

result = df_tmt

In [ ]:
spk_width = {}
tr2peak = {}
neuron_type = {}
tp_dic = {}
w_dic = {}
ls = []
ls2 = []
ls3 = []

num_units = result['cuid'].nunique()

for ii,cuid in enumerate(result.cuid.unique()[:]):    
    if ii%200 == 0:
        print('done with {0} out of {1}'.format(ii, num_units))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    tmt_data = np.array(result[(result['cuid'] == cuid)].tmt)

    y = resampy.resample( tmt_data[::-1] , 1 ,10,  filter='sinc_window',
                                    num_zeros=10, precision=5, window=ssig.hann)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #trough-to-peak
    trough_idx = y.argmin()
    peak_idx = y[:y.argmin()].argmax()
    tp = abs((trough_idx - peak_idx)/300.0)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    x = np.arange(y.size)
    y_gaus = y*(-1)
    popt,pcov = scipy.optimize.curve_fit(gaus,x,y_gaus,p0=[0.2, y.argmin(), 10])
    fwhm = popt[-1]/300*2.355
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #width of spike    
    f,pxx = ssig.welch(tmt_data, fs=30000,  nfft=5096,  nperseg=48,
                          return_onesided=True, scaling='spectrum')

    df = np.vstack((f, pxx))
    df = pd.DataFrame(df)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    idx = df.T[1].idxmax()
    if not np.isnan(idx):
        w = df.T[0][idx]
        w = 1/w*1000.0
        ls.append(tp)
        ls2.append(w)
        spk_width[cuid] = w
        tr2peak[cuid] = tp
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if tp<=0.45: #can add something for width here as well, but then you'll have 'fs', 'rs', and 'un' units
            neuron_type[cuid] = 'fs'
        elif tp>0.45:
            neuron_type[cuid] = 'rs'
        else:
            neuron_type[cuid] = 'un'
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # p/t ratio
        ptr = result[(result['cuid'] == cuid)]['tmt'].max()/result[(result['cuid'] == cuid)]['tmt'].min()
        ls3.append(abs(ptr))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~        
df_tmt['n_type'] = df_tmt.cuid.map(neuron_type)
df_tmt['trough2peak'] = df_tmt.cuid.map(tr2peak)
df_tmt['spk_width'] = df_tmt.cuid.map(spk_width)

df_tmt.head()

In [ ]:
print('Number of rs units: {0}'.format(df_tmt[df_tmt['n_type'] == 'rs'].cuid.nunique()))
print('Number of fs units: {0}'.format(df_tmt[df_tmt['n_type'] == 'fs'].cuid.nunique()))
print('Number of un units: {0}'.format(df_tmt[df_tmt['n_type'] == 'un'].cuid.nunique()))

# Reorder the columns

In [ ]:
# just a last reordering of the columns for easy viewing -- spikes df
cols = ['cluster_id', 'spikes', 'trial', 'stim_id', 'trial_spikes', 
        'depth', 'cuid', 'situ', 'group', 'region', 'et', 'cc','path']

df_spikes = df_spikes[cols]

df_spikes.head()

In [ ]:
# just a last reordering of the columns for easy viewing -- waveforms df
cols = ['tmt', 'n_type', 'trough2peak', 'spk_width', 'stim_id', 'cuid', 
        'depth', 'region', 'situ', 'group', 'cc', 'et', 'path']

df_tmt = df_tmt[cols]

df_tmt.head()

# Save the Spikes and Waveform dataFrames

In [ ]:

df_spikes.to_pickle(r"D:\mz_Data\saved_dfs\HPC_nmda\spikes_df.pkl")

df_tmt.to_pickle(r"D:\mz_Data\saved_dfs\HPC_nmda\waveforms_df.pkl")


---

# Making the PSTH dataFrame

In [ ]:
df_rez.head()

In [ ]:
ls_psth = []
i=0

num_units = df_rez['cuid'].nunique()

for iii, unit in enumerate(df_rez['cuid'].unique()): ##### I changed this from df_rez to data_df to check the units
    cuid = str(unit)
    tmp2 = df_rez[(df_rez.cuid == unit)]             ##### I changed this from df_rez to data_df to check the units
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    cluster_id = tmp2.cluster_id.values[0]
    situ = tmp2.situ.values[0]
    group = tmp2.group.values[0]
    et = tmp2.et.values[0]
    cc = tmp2.cc.values[0]
    path = tmp2.path.values[0]
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    try:
        tmt, depth, ch_idx = mz_ena.ksort_get_tmt(tmp2, cluster_id, templates, channel_groups)
    except:
        i = i+1        
        continue    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    if iii%200 == 0: #think of this as a loading bar to know for far it has run
        print('done with {0} out of {1}'.format(iii, num_units))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    for stim_id in tmp2.stim.unique():
        tmp3=tmp2[tmp2.stim==stim_id]
        df = mz_ena.getRaster_kilosort(tmp3, unit, trial_length) 
        trials_number_not_empty = len(df.trial.unique())    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        h, ttr = mz_ena.PSTH(df.times, th_bin, trial_length, trials_number_not_empty)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        zscore = sstat.mstats.zscore(h)
        mean = np.mean(h[0:50])#The Baseline period. Be sure it matches time course of experiments##
        if mean<=0:
            std=1
        else:
            std = np.std(h[0:50])#The Baseline period. Be sure it matches time course of experiments##
        ztc = (h - mean)/std
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        df_psth_tmp = pd.DataFrame({'times':ttr,
                                    'stim_id': stim_id,
                                    'Hz':h,
                                    'cluster_id': cluster_id,
                                    'depth': depth,
                                    'zscore':zscore, 
                                    'ztc':ztc,
                                    'situ':situ,
                                    'group':group,
                                    'et':et,
                                    'cc':cc,
                                    'cuid':cuid,
                                    'path':path})
        ls_psth.append(df_psth_tmp)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
df_psth = pd.concat(ls_psth)
print("Total errors: {0} out of {1} units".format(i,num_units))
print("ALL DONE!")


In [ ]:
print('Min unit depth on probe:', df_psth['depth'].min())
print('Max unit depth on probe:',df_psth['depth'].max())

# print('\n',np.sort(df_psth['depth'].unique()))

In [ ]:
null_df = df_psth[df_psth['zscore'] < 20]

print('# good units: %d' % df_psth['cuid'].nunique())
print('# good units w/ z-score < 20: %d' % null_df['cuid'].nunique())

---

# Updating the PSTH dataFrame with metadata

In [ ]:
# null_df['group'] = null_df.apply(lambda row: label_group(row), axis=1)
null_df['region'] = null_df.apply(lambda row: label_region(row), axis=1)

null_df.head()

---

# Reordering the columns

In [ ]:
# just a last reordering of the columns for easy viewing -- Reward trials
cols = ['stim_id', 'times', 'cuid', 'depth', 'Hz', 'zscore', 'ztc', 
        'region', 'situ', 'group', 'cc', 'et', 'cluster_id', 'path']
null_df = null_df[cols]
null_df.head()

---

# Save the dataframe and plot elsewhere

In [ ]:


null_df.to_pickle(r"D:\mz_Data\saved_dfs\HPC_nmda\psth_df.pkl")



---